In [17]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [18]:
data = pd.read_excel('../data/The history of global natural gas production.xlsx',sheet_name = 1)

In [19]:
data = data.melt(id_vars=['Year'],var_name='country',value_name='gas' )

In [20]:
data.to_csv('../data/tidy_gas.csv')

In [21]:
data = pd.read_csv('../data/tidy_gas.csv',index_col=0)
country_border = gpd.read_file('../data/countries.geojson')

In [22]:
country_border = country_border.rename(columns={'ADMIN':'country'})

In [23]:
data_with_border = data.merge(country_border,how='left',on='country')

In [24]:
data_with_border[data_with_border['ISO_A2'].isna()]['country'].unique()

array(['United States', 'Congo', "Cote d'Ivoire", 'Czechia',
       'Czechoslovakia', 'Serbia', 'Tanzania', 'USSR', 'World',
       'Yugoslavia'], dtype=object)

In [25]:
found = []
not_found=[]
for j in data_with_border[data_with_border['ISO_A2'].isna()]['country'].unique():
    find = 0
    print('searching for any country name containing {}…………\n'.format(j))
    for i in country_border['country'].unique():
        if j in i :
            print('yes there is {}\n'.format(i))
            find = 1
            found.append((i,j))
    if find == 0:
        print('no there is  no {}\n'.format(j))
        not_found.append(j)
print('countries below are not found')
print(not_found)

searching for any country name containing United States…………

yes there is United States Minor Outlying Islands

yes there is United States of America

yes there is United States Virgin Islands

searching for any country name containing Congo…………

yes there is Democratic Republic of the Congo

yes there is Republic of Congo

searching for any country name containing Cote d'Ivoire…………

no there is  no Cote d'Ivoire

searching for any country name containing Czechia…………

no there is  no Czechia

searching for any country name containing Czechoslovakia…………

no there is  no Czechoslovakia

searching for any country name containing Serbia…………

yes there is Republic of Serbia

searching for any country name containing Tanzania…………

yes there is United Republic of Tanzania

searching for any country name containing USSR…………

no there is  no USSR

searching for any country name containing World…………

no there is  no World

searching for any country name containing Yugoslavia…………

no there is  no

In [26]:
found

[('United States Minor Outlying Islands', 'United States'),
 ('United States of America', 'United States'),
 ('United States Virgin Islands', 'United States'),
 ('Democratic Republic of the Congo', 'Congo'),
 ('Republic of Congo', 'Congo'),
 ('Republic of Serbia', 'Serbia'),
 ('United Republic of Tanzania', 'Tanzania')]

In [27]:
correct_indice = [1,4,5,6]
found = [found[i]for i in correct_indice]

In [28]:
found

[('United States of America', 'United States'),
 ('Republic of Congo', 'Congo'),
 ('Republic of Serbia', 'Serbia'),
 ('United Republic of Tanzania', 'Tanzania')]

In [29]:
data_with_border[data_with_border['country'] == 'United States']

,Year,country,gas,ISO_A3,ISO_A2,geometry
0,1900,United States,0.26582551,NaN,NaN,None
1,1901,United States,0.29458103,NaN,NaN,None
2,1902,United States,0.32333653,NaN,NaN,None
3,1903,United States,0.35209204,NaN,NaN,None
4,1904,United States,0.38084755,NaN,NaN,None
...,...,...,...,...,...,...
118,2018,United States,30.2736996,NaN,NaN,None
119,2019,United States,33.411996,NaN,NaN,None
120,2020,United States,32.979762,NaN,NaN,None
121,2021,United States,33.9891084,NaN,NaN,None


In [30]:
for i,j in found:
    data_with_border.loc[data_with_border['country'] == j,['ISO_A3','ISO_A2','geometry']] = country_border[country_border['country'] ==i ][['ISO_A3','ISO_A2','geometry']].values

In [31]:
data_with_border[data_with_border['ISO_A2'].isna()]['country'].unique()

array(["Cote d'Ivoire", 'Czechia', 'Czechoslovakia', 'USSR', 'World',
       'Yugoslavia'], dtype=object)

In [16]:
data_with_border_no_border = data_with_border[data_with_border['ISO_A2'].isna()]
data_with_border = data_with_border[~data_with_border['ISO_A2'].isna()]

In [ ]:
#转化为geopandas数据格式
data_with_border = gpd.GeoDataFrame(data_with_border,geometry='geometry')

In [ ]:
#以下数据转化时间太久，跑了十分钟都没跑出来，中间中断后发现未完成的文件就有1.74GB。
#这就不存储为文件了，改为每次运行时做以上数据清洗。
#data_with_border.to_file('../data/tidy_data_with_border.geojson',driver="GeoJSON")